# Operation IceBridge SnowRadar (2-8 GHz) Example

Example of using wavelet picker with CRESIS 2-8 GHz snow radar data (MATFILE)

In [ ]:
#pySnowRadar modules
from pySnowRadar import SnowRadar
from pySnowRadar.processing import extract_layers
from pySnowRadar.algorithms import Wavelet_TN, Peakiness

#Community packages
import os
import datetime
import time
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
radar_file = "../pySnowRadar/data/sr/Data_20160419_04_010.mat"
radar_dat = SnowRadar(radar_file, l_case='full')

In [ ]:
# Find surface and trim data
radar_dat.surf_bin, radar_dat.surface = radar_dat.get_surface()
radar_bounds = radar_dat.get_bounds(2,1)
radar_dat.plot_quicklook(ylim = list(radar_bounds))

## Application of wavelet picker

In [ ]:
# Set up picker parameters and extract interfaces using the Newman wavelet
params={'snow_density':0.3,
        'ref_snow_layer': 1,
        'cwt_precision': 10}

res_TN = extract_layers(radar_file, picker=Wavelet_TN, params=params, dump_results=False)

In [ ]:
# Plot interface picks and snow depth
fig, (ax1, ax2) = plt.subplots(nrows=2, figsize=(10,8), sharex=True)
ax1.imshow(10 * np.log10(radar_dat.data_radar/np.nanmax(radar_dat.data_radar)), interpolation='none', cmap="gist_gray",  aspect='auto')
ax1.plot(res_TN['b_as'], linestyle='--', color='b',alpha=0.9,label='A-S')
ax1.plot(res_TN['b_si'], linestyle='--', color='r',alpha=0.9,label='S-I')
ax1.legend(loc="upper right")


# Rolling average of depth over 5 bins
N = 5
snow_depth_rolling = np.convolve(res_TN['snow_depth'], np.ones((N,))/N, mode='valid')
estart = N/2
eend = len(snow_depth_rolling)+(N/2)

ax2.scatter(np.arange(0,len(res_TN['snow_depth'])), res_TN['snow_depth'], color = 'k', s = 1)
ax2.plot(np.arange(estart, eend),snow_depth_rolling, color='r')


ax1.set_ylim(radar_bounds)
ax2.set_xlim(0, len(res_TN['snow_depth']))
ax1.set_ylabel("Range bin [-]")
ax2.set_ylabel("Snow depth (m)")
ax2.set_xlabel("Radar trace [-]")

In [ ]:
col_idx = np.random.randint(0,radar_dat.data_radar.shape[1])
fig, ax = plt.subplots(figsize=(14,5))
with np.errstate(divide='ignore'):
    ax.plot(10*np.log10(radar_dat.data_radar[:,col_idx]), color = 'k', label=None)


    
# Air-Snow picked interface
ax.vlines(res_TN['b_as'][col_idx], 0, 1, label='A-S', linestyles='dashed', transform=ax.get_xaxis_transform(), color='r')
# Snow-Ice picked interface
ax.vlines(res_TN['b_si'][col_idx], 0, 1, label='A-S', linestyles='dashed', transform=ax.get_xaxis_transform(), color='b')

ax.set_xlim(0, len(radar_dat.data_radar[:,col_idx]))
ax.set_xlabel("Fast time bin")
ax.set_ylabel("Amplidtude (dB)")
ax.legend(loc="upper right")